In [1]:
import os
import re
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt



In [2]:
# 1. 데이터 수집하기

data_path = "ChatbotData .csv"  
data = pd.read_csv(data_path)
print(f"data.shape: {data.shape}")
print(data.head())


data.shape: (11823, 3)
                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0


In [3]:
# 2. 데이터 전처리하기
# Q, A 컬럼을 리스트 형태로 저장
questions = data['Q'].tolist()
answers   = data['A'].tolist()

print("샘플 질문 :", questions[0])
print("샘플 답변 :", answers[0])


샘플 질문 : 12시 땡!
샘플 답변 : 하루가 또 가네요.


In [4]:
# 전처리 함수 
def preprocess_sentence(sentence):
    # 양쪽 공백 제거
    sentence = sentence.strip()
    # 구두점(?.!,) 앞뒤에 공백 추가
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'\s+', ' ', sentence)
    # 한글, 숫자, 영문, 그리고 구두점(?.!,)를 제외한 모든 문자를 공백으로 대체
    sentence = re.sub(r"[^ㄱ-ㅎ가-힣0-9a-zA-Z?.!,]+", " ", sentence)
    sentence = sentence.strip()
    return sentence


# 전처리 후 샘플 확인
print("전처리 후의 샘플 질문 :", preprocess_sentence(questions[0]))
print("전처리 후의 샘플 답변 :", preprocess_sentence(answers[0]))
# 전체 데이터에 전처리 적용
questions = [preprocess_sentence(q) for q in questions]
answers   = [preprocess_sentence(a) for a in answers]



전처리 후의 샘플 질문 : 12시 땡 !
전처리 후의 샘플 답변 : 하루가 또 가네요 .


In [5]:
# 3. SubwordTextEncoder 사용하여 단어장(Vocabulary) 만들기
# 질문과 답변 모두를 사용해 토크나이저를 구성합니다.
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)
VOCAB_SIZE = tokenizer.vocab_size + 2  
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

print("단어장 크기:", VOCAB_SIZE)



단어장 크기: 8166


In [6]:
# 4. 정수 인코딩 및 패딩
# 샘플의 최대 길이
MAX_LENGTH = 40
print("MAX_LENGTH:", MAX_LENGTH)

def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []
    for (sentence1, sentence2) in zip(inputs, outputs):
        # 정수 인코딩 시 START, END 토큰 추가
        sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
        sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

        # 최대 길이 이하인 문장만 선택
        if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
            tokenized_inputs.append(sentence1)
            tokenized_outputs.append(sentence2)
    
    tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
    tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
        tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
    
    return tokenized_inputs, tokenized_outputs


questions, answers = tokenize_and_filter(questions, answers)
print("필터링 후의 질문 샘플 개수:", len(questions))
print("필터링 후의 답변 샘플 개수:", len(answers))


MAX_LENGTH: 40
필터링 후의 질문 샘플 개수: 11823
필터링 후의 답변 샘플 개수: 11823


In [7]:
# 5. tf.data.Dataset 구성 (교사 강요 적용)
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더 입력은 answers의 [:-1], 출력(레이블)은 [1:] 사용
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1]
    },
    {
        'outputs': answers[:, 1:]
    }
))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


In [8]:
# 6. 모델 구성하기 (Transformer)
# 아래는 Transformer 모델에 필요한 각 레이어 및 함수들을 정의한 코드입니다.

# 6-1. 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)
    
    def get_angles(self, position, i, d_model):
        angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return position * angles
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
            i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
            d_model=d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        pos_encoding = tf.stack([sines, cosines], axis=0)
        pos_encoding = tf.transpose(pos_encoding, [1, 2, 0])
        pos_encoding = tf.reshape(pos_encoding, [position, d_model])
        pos_encoding = pos_encoding[tf.newaxis, ...]
        return tf.cast(pos_encoding, tf.float32)
    
    def call(self, inputs):
        return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]



In [9]:
# 6-2. 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
    matmul_qk = tf.matmul(query, key, transpose_b=True)
    depth = tf.cast(tf.shape(key)[-1], tf.float32)
    logits = matmul_qk / tf.math.sqrt(depth)
    if mask is not None:
        logits += (mask * -1e9)
    attention_weights = tf.nn.softmax(logits, axis=-1)
    output = tf.matmul(attention_weights, value)
    return output



In [10]:
# 6-3. 멀티 헤드 어텐션 레이어
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads
        self.query_dense = tf.keras.layers.Dense(units=d_model)
        self.key_dense = tf.keras.layers.Dense(units=d_model)
        self.value_dense = tf.keras.layers.Dense(units=d_model)
        self.dense = tf.keras.layers.Dense(units=d_model)
    
    def split_heads(self, inputs, batch_size):
        inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(inputs, perm=[0, 2, 1, 3])
    
    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
        batch_size = tf.shape(query)[0]
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        outputs = self.dense(concat_attention)
        return outputs



In [11]:
# 6-4. 마스크 생성 함수들
def create_padding_mask(x):
    mask = tf.cast(tf.math.equal(x, 0), tf.float32)
    return mask[:, tf.newaxis, tf.newaxis, :]


def create_look_ahead_mask(x):
    seq_len = tf.shape(x)[1]
    look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
    padding_mask = create_padding_mask(x)
    return tf.maximum(look_ahead_mask, padding_mask)



In [12]:
# 6-5. 인코더의 레이어 정의
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    
    attention = MultiHeadAttention(d_model, num_heads, name="attention")({
        'query': inputs,
        'key': inputs,
        'value': inputs,
        'mask': padding_mask
    })
    attention = tf.keras.layers.Dropout(rate=dropout)(attention)
    attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)
    
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention + outputs)
    
    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)



In [13]:
# 6-6. 인코더 전체 정의
def encoder(vocab_size, num_layers, units, d_model, num_heads, dropout, name="encoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
    
    for i in range(num_layers):
        outputs = encoder_layer(units, d_model, num_heads, dropout, name=f"encoder_layer_{i}")([outputs, padding_mask])
    
    return tf.keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)



In [14]:
# 6-7. 디코더의 한 레이어 정의
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    
    attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")({
        'query': inputs,
        'key': inputs,
        'value': inputs,
        'mask': look_ahead_mask
    })
    attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1 + inputs)
    
    attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")({
        'query': attention1,
        'key': enc_outputs,
        'value': enc_outputs,
        'mask': padding_mask
    })
    attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2 + attention1)
    
    outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = tf.keras.layers.Dense(units=d_model)(outputs)
    outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
    outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs + attention2)
    
    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name
    )


# 6-8. 디코더 전체 정의
def decoder(vocab_size, num_layers, units, d_model, num_heads, dropout, name="decoder"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = tf.keras.Input(shape=(1, None, None), name="look_ahead_mask")
    padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")
    
    embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
    embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
    outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)
    
    for i in range(num_layers):
        outputs = decoder_layer(units, d_model, num_heads, dropout, name=f"decoder_layer_{i}")(
            [outputs, enc_outputs, look_ahead_mask, padding_mask])
    
    return tf.keras.Model(
        inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
        outputs=outputs,
        name=name
    )



In [15]:
# 6-9. Transformer 모델 정의
def transformer(vocab_size, num_layers, units, d_model, num_heads, dropout, name="transformer"):
    inputs = tf.keras.Input(shape=(None,), name="inputs")
    dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")
    
    # 인코더 마스크
    enc_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='enc_padding_mask')(inputs)
    
    # 디코더의 룩 어헤드 마스크 (미래 토큰 마스킹 + 패딩 마스크)
    look_ahead_mask = tf.keras.layers.Lambda(
        create_look_ahead_mask, output_shape=(1, None, None),
        name='look_ahead_mask')(dec_inputs)
    
    # 디코더의 두 번째 어텐션에서 사용할 인코더 출력 마스킹
    dec_padding_mask = tf.keras.layers.Lambda(
        create_padding_mask, output_shape=(1, 1, None),
        name='dec_padding_mask')(inputs)
    
    enc_outputs = encoder(vocab_size, num_layers, units, d_model, num_heads, dropout)(
        [inputs, enc_padding_mask])
    dec_outputs = decoder(vocab_size, num_layers, units, d_model, num_heads, dropout)(
        [dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])
    
    outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
    return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)


In [16]:
# 6-10. 모델 생성 및 요약
tf.keras.backend.clear_session()

# 하이퍼파라미터 설정
NUM_LAYERS = 2      # 인코더/디코더 층 개수
D_MODEL = 256       # 모델 차원
NUM_HEADS = 8       # 멀티 헤드 어텐션의 헤드 수
UNITS = 512         # 피드포워드 신경망 내부 차원
DROPOUT = 0.1       # 드롭아웃 비율

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

# 7. 손실 함수 및 커스텀 학습률 스케줄러
def loss_function(y_true, y_pred):
    y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
    loss = tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')(y_true, y_pred)
    mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
    loss = loss * mask
    return tf.reduce_mean(loss)


class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        self.warmup_steps = warmup_steps
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)
        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)




Model: "transformer"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
dec_inputs (InputLayer)         [(None, None)]       0                                            
__________________________________________________________________________________________________
enc_padding_mask (Lambda)       (None, 1, 1, None)   0           inputs[0][0]                     
__________________________________________________________________________________________________
encoder (Functional)            (None, None, 256)    3144704     inputs[0][0]                     
                                                                 enc_padding_mask[0][0] 

In [28]:
# 8. 모델 컴파일 및 학습 준비
learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# 9. 모델 학습하기
EPOCHS = 40
model.fit(dataset, epochs=EPOCHS, verbose=1)


Epoch 1/40
185/185 [==============================] - 15s 54ms/step - loss: 0.0274 - accuracy: 0.1689
Epoch 2/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0173 - accuracy: 0.1712
Epoch 3/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0105 - accuracy: 0.1729
Epoch 4/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0074 - accuracy: 0.1736
Epoch 5/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0064 - accuracy: 0.1738
Epoch 6/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0064 - accuracy: 0.1738
Epoch 7/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0067 - accuracy: 0.1736
Epoch 8/40
185/185 [==============================] - 10s 53ms/step - loss: 0.0070 - accuracy: 0.1736
Epoch 9/40
185/185 [==============================] - 10s 54ms/step - loss: 0.0072 - accuracy: 0.1735
Epoch 10/40
185/185 [==============================] - 10s 54ms/step - loss: 0.008

In [29]:
# 10. 챗봇 테스트 
def decoder_inference(sentence):
    
    sentence = preprocess_sentence(sentence)
    sentence = tf.expand_dims(START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)
    # 디코더 입력 초기값 (START_TOKEN)
    output_sequence = tf.expand_dims(START_TOKEN, 0)
    
    for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output_sequence], training=False)
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
        if tf.equal(predicted_id, END_TOKEN[0]):
            break
        output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)
    
    return tf.squeeze(output_sequence, axis=0)


def sentence_generation(sentence):
    prediction = decoder_inference(sentence)
    predicted_sentence = tokenizer.decode([i for i in prediction.numpy() if i < tokenizer.vocab_size])
    print('입력 : {}'.format(sentence))
    print('출력 : {}'.format(predicted_sentence))
    return predicted_sentence



In [34]:
sentence_generation("안녕하세요")

sentence_generation("오늘 날씨가 좋아요")

sentence_generation("반가워요")

sentence_generation("저녁 뭐먹죠")

sentence_generation("주말에 만나서 밥이나 먹자")


입력 : 안녕하세요
출력 : 안녕하세요 .
입력 : 오늘 날씨가 좋아요
출력 : 집밖에 나가기가 힘들것 같아요 .
입력 : 반가워요
출력 : 반갑습니다 .
입력 : 저녁 뭐먹죠
출력 : 맛있는 거 드세요 .
입력 : 주말에 만나서 밥이나 먹자
출력 : 밥심으로 사는 거죠 .


'밥심으로 사는 거죠 .'

## 회고


트랜스포머는 이름만 들어봤던 개념이라 처음 접했을 때 너무 어렵게 느껴졌다. 
Self-Attention과 Multi-Head Attention 같은 개념이 낯설었지만, 2일 동안 학습하며 조금씩 구조와 작동 원리를 이해하려고 노력했다
완벽하지는 않지만, 이 노력한 과정이 유익하고 보람 있었으면 좋겠다
에포크는 40번 돌려봤는데 5번만 돌려도 충분한거같다 